### Trabalho Prático 2 – Soluções para problemas difíceis

#### Importações

In [91]:
import numpy as np
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo
import time
import warnings
from itertools import cycle, islice
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_rand_score
import os

 #### Distância de Minkowski

In [ ]:
# p = 1 (Distância Manhattan)
# p = 2 (Distância Euclidiana)

def minkowski(x, y, p):
   return np.power(np.sum(np.abs(x - y) ** p), 1/p)

#### Cálculo do raio da solução

In [84]:
def raio(labels, centros, X):
    raio = -1
    for i in range(len(labels)):
        # Usando indexação direta para acessar X, que agora é um array NumPy
        distancia = minkowski(X[i], centros[labels[i]], p=2)
        raio = max(raio, distancia)
    return raio

#####K-means

In [218]:
def kmeans_exec(X, y, k, num_execucoes=30):
    tmp_exec_kmeans = []
    silhueta_kmeans = []
    indice_rand_kmeans = []
    raio_exec_kmeans = []

    # Calcular a matriz de distâncias entre os centros apenas uma vez
    centros_iniciais = KMeans(n_clusters=k, n_init=1).fit(X).cluster_centers_
    distancias_centros = np.array([min([minkowski(c, p, p=1) for c in centros_iniciais]) for p in X.values])

    for i in range(num_execucoes):
        start_time = time.time()

        kmeans = KMeans(n_clusters=k, random_state=i, n_init=10)
        labels_kmeans = kmeans.fit_predict(X)

        end_time = time.time()
        execution_time = end_time - start_time

        tmp_exec_kmeans.append(execution_time)
        silhueta_kmeans.append(silhouette_score(X, labels_kmeans))
        indice_rand_kmeans.append(adjusted_rand_score(y, labels_kmeans))

        centros_kmeans = kmeans.cluster_centers_
        raio_max_kmeans = raio(labels_kmeans, centros_kmeans, distancias_centros)
        raio_exec_kmeans.append(raio_max_kmeans)

    # Retornar as médias e desvios-padrão das métricas
    resultados_kmeans = {
        'Algoritmo': 'K-Means',
        'Tempo Execução (s)': f"{np.mean(tmp_exec_kmeans):.4f} ± {np.std(tmp_exec_kmeans):.4f}",
        'Silhueta': f"{np.mean(silhueta_kmeans):.4f} ± {np.std(silhueta_kmeans):.4f}",
        'Índice de Rand Ajustado': f"{np.mean(indice_rand_kmeans):.4f} ± {np.std(indice_rand_kmeans):.4f}",
        'Raio Máximo': f"{np.mean(raio_exec_kmeans):.4f} ± {np.std(raio_exec_kmeans):.4f}"
    }

    return resultados_kmeans


####Processamento dos dados sinteticos

In [200]:
def carregar_datasets(path, num_datasets, sintetico):
    datasets = []
    for i in range(num_datasets):
        if(sintetico ==1):
          filename = os.path.join(path, f'dataset_{i}.csv')
        elif(sintetico ==2):
          filename = os.path.join(path, f'dataset_{i}_dist_mult.csv')

        df = pd.read_csv(filename)
        datasets.append(df)
    return datasets

def processar_datasets_kmeans(datasets, sintetico):
    resultados = []
    for i, df in enumerate(datasets):

      if(sintetico ==1):
        X = df[['feature_1', 'feature_2']]
        y = df['target']

      elif(sintetico ==2):
        X = df[['X1', 'X2']]
        y = df['Center']

      num_clusters = y.nunique()
      resultado_kmeans = kmeans_exec(X, y, num_clusters)

      if(sintetico ==1):
        resultado_kmeans['Dataset'] = f'dataset_{i}.csv'
      elif(sintetico ==2):
        resultado_kmeans['Dataset'] = f'dataset_{i}_dist_mult.csv'

      resultados.append(resultado_kmeans)
    return pd.DataFrame(resultados)


In [219]:
def datasetUCI(n, data_set_name):

  dataset = fetch_ucirepo(id=n)
  X = dataset.data.features
  y = dataset.data.targets.squeeze()

  if(n ==1):
    X = X.drop('Sex', axis = 1)

  X = X.dropna()
  y = y.loc[X.index]
  k = y.nunique()

  resultados_kmeans = kmeans_exec(X, y, k)

  resultados_df = pd.DataFrame([resultados_kmeans])
  resultados_df.to_csv(f'resultados_kmeans_UCI_{data_set_name}.csv', index=False)

In [221]:
def main_Kmeans():
  #UCI
  datasetUCI(850, 'Raisin')
  datasetUCI(50, 'image_segmentation')
  datasetUCI(863, 'maternal_health_risk')
  datasetUCI(161, 'mammographic_mass')
  datasetUCI(80, 'optical_recognition_of_handwritten_digits')
  datasetUCI(149, 'statlog_vehicle_silhouettes')
  datasetUCI(110, 'yeast')
  datasetUCI(176, 'blood_transfusion_service_center')
  datasetUCI(1, 'abalone')
  datasetUCI(329, 'diabetic_retinopathy_debrecen')

  ##Sinteticos 1
  datasets = carregar_datasets('.', 10, 1)
  resultados_kmeans = processar_datasets_kmeans(datasets, 1)
  resultados_kmeans.to_csv('resultados_kmeans_sinteticos_1.csv', index=False)

  ##Sinteticos 2
  datasets = carregar_datasets('.', 10, 2)
  resultados_kmeans = processar_datasets_kmeans(datasets, 2)
  resultados_kmeans.to_csv('resultados_kmeans_sinteticos_2.csv', index=False)

# Executa a função principal
if __name__ == "__main__":
    main_Kmeans()